In [23]:
import numpy as np
a = np.arange(9).reshape(3,3)
a.diagonal(0)

array([0, 4, 8])

In [32]:

def calc_projection_matrix(source, weights):
    proj = np.zeros_like(source)
    N = source.shape[0]

    # 0 deg
    proj[0,:] = np.sum(source, ax=1)

    # 90
    proj[2,:] = np.sum(source, ax=1)

    # diag
    proj[1,:] = np.array([np.sum(proj.diagonal(n)) for n in range(-N//2 - 1, N//2 + 1)]) @ weights
    source = np.flip(source, 0)
    source = np.flip(source, 1)
    proj[3,:] = np.array([np.sum(proj.diagonal(n)) for n in range(-N//2 - 1, N//2 + 1)]) @ weights


In [327]:
weights = np.array([[14,0,0,0,0],
[88,1,0,0,0],
[39,61,0,0,0],
[0,75,25,0,0],
[0,4,91,4,0],
[0,0,25,75,0],
[0,0,0,61,39],
[0,0,0,1,88],
[0,0,0,0,14]])/100

In [328]:
source = np.array([
[0,0,0,0,2],
[0,9,0,0,0],
[0,0,0,0,0],
[0,7,0,8,0],
[0,0,0,0,0]])

In [329]:
def calc_projection_matrix(source, weights):
    proj = np.zeros((4, 5))
    N = source.shape[0]

    # 0
    proj[0,:] = np.sum(source, axis=0)

    # 90
    proj[2,:] = np.flip(np.sum(source, axis=1))

    # diag
    proj[1,:] = np.array(np.array([np.sum(source.diagonal(n)) for n in range(-N//2 - 1, N//2 + 3)])) @ weights
    source = np.flip(source, 0)

    proj[3,:] = np.flip([np.sum(source.diagonal(n)) for n in range(-N//2 - 1, N//2 + 3)] @ weights)
    return proj


In [396]:
proj =calc_projection_matrix(source, weights)

In [402]:
print(bmatrix(proj))

\begin{pmatrix}
  0. & 16. & 0. & 8. & 2.\\
  2.73 & 4.95 & 15.47 & 0.68 & 0.28\\
  0. & 15. & 0. & 9. & 2.\\
  3.12 & 5.24 & 8.19 & 5.85 & 3.51\\
\end{pmatrix}


In [403]:

def convolute_projection(proj):
    filt = np.array([-.1, .25, -.1])
    conv = np.zeros_like(proj)
    for row in range(0, proj.shape[0]):
        conv[row] = np.convolve(proj[row], filt)[1:-1]
    return conv

In [404]:
conv = convolute_projection(proj)

In [405]:
print(bmatrix(conv))

\begin{pmatrix}
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
  0.188 & -0.583 & 3.304 & -1.405 & 0.002\\
  -1.5 & 3.75 & -2.4 & 2.05 & -0.4\\
  0.256 & 0.179 & 0.938 & 0.293 & 0.292\\
\end{pmatrix}


In [406]:
backweights = np.array([
    [27, 100, 72, 0, 0, 0, 0, 0, 0],
    [0, 1, 93, 99, 7, 0, 0, 0, 0],
    [0, 0, 0, 50, 100, 50, 0, 0, 0],
    [0, 0, 0, 0, 7, 99, 93, 1, 0],
    [0, 0, 0, 0, 0, 0, 73, 100, 27]])/100

In [407]:

def reconstruct_diags(proj, weights):
   return proj[1] @ weights, proj[3] @ weights
   
def reproject_diagonal(diag):
    N = diag.size
    d = int((diag.size + 1)/2)
    rep = np.zeros((d, d))

    for di, elem in enumerate(diag):
        offset = int(d - di) - 1
        for i in range(0, d - int(np.abs(offset))):
            if offset >= 0:
                rep[i + offset][i] = elem
            else:
                rep[i][i - offset] = elem

    return rep

def reproject_straight(row):
    d = row.size
    rep = np.empty((d, d))

    for i, elem in enumerate(row):
        rep[:, i] = elem
    return rep

def reconstruct(proj, backweights):
    p0, p90 = proj[0], proj[2]  # uuuugly
    p45, p135 = reconstruct_diags(proj, backweights)

    r0 = reproject_straight(p0)
    r90 = np.flip(reproject_straight(p90).T)
    r45 = reproject_diagonal(p45)
    r135 = np.flip(reproject_diagonal(p135).T, 1)
    print(bmatrix(r0))
    print(bmatrix(r45))
    print(bmatrix(r90))
    print(bmatrix(r135))
    return r0 + r90 + 1/2*(r45 + r135)
        
def bmatrix(a, prec=3):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    with np.printoptions(precision=prec, suppress=True):
        if len(a.shape) > 2:
            raise ValueError('bmatrix can at most display two dimensions')
        lines = str(a).replace('[', '').replace(']', '').splitlines()
        rv = [r'\begin{pmatrix}']
        rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
        rv +=  [r'\end{pmatrix}']
        return '\n'.join(rv)

In [409]:
print(bmatrix(reconstruct(proj, backweights)))


\begin{pmatrix}
  0. & 16. & 0. & 8. & 2.\\
  0. & 16. & 0. & 8. & 2.\\
  0. & 16. & 0. & 8. & 2.\\
  0. & 16. & 0. & 8. & 2.\\
  0. & 16. & 0. & 8. & 2.\\
\end{pmatrix}
\begin{pmatrix}
  15.864 & 8.408 & 0.837 & 0.287 & 0.076\\
  12.636 & 15.864 & 8.408 & 0.837 & 0.287\\
  6.569 & 12.636 & 15.864 & 8.408 & 0.837\\
  2.78 & 6.569 & 12.636 & 15.864 & 8.408\\
  0.737 & 2.78 & 6.569 & 12.636 & 15.864\\
\end{pmatrix}
\begin{pmatrix}
  2. & 2. & 2. & 2. & 2.\\
  9. & 9. & 9. & 9. & 9.\\
  0. & 0. & 0. & 0. & 0.\\
  15. & 15. & 15. & 15. & 15.\\
  0. & 0. & 0. & 0. & 0.\\
\end{pmatrix}
\begin{pmatrix}
  0.842 & 3.172 & 7.12 & 9.283 & 8.966\\
  3.172 & 7.12 & 9.283 & 8.966 & 9.886\\
  7.12 & 9.283 & 8.966 & 9.886 & 8.003\\
  9.283 & 8.966 & 9.886 & 8.003 & 3.569\\
  8.966 & 9.886 & 8.003 & 3.569 & 0.948\\
\end{pmatrix}
\begin{pmatrix}
  10.353 & 23.79 & 5.978 & 14.785 & 8.521\\
  16.904 & 36.492 & 17.845 & 21.902 & 16.087\\
  6.844 & 26.959 & 12.415 & 17.147 & 6.42\\
  21.031 & 38.768 & 26.26

In [410]:
print(bmatrix(reconstruct(conv, backweights)))

\begin{pmatrix}
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
  -1.6 & 4. & -2.4 & 1.8 & -0.3\\
\end{pmatrix}
\begin{pmatrix}
  3.165 & 0.261 & -1.305 & -0.012 & 0.001\\
  1.076 & 3.165 & 0.261 & -1.305 & -0.012\\
  -0.407 & 1.076 & 3.165 & 0.261 & -1.305\\
  0.182 & -0.407 & 1.076 & 3.165 & 0.261\\
  0.051 & 0.182 & -0.407 & 1.076 & 3.165\\
\end{pmatrix}
\begin{pmatrix}
  -0.4 & -0.4 & -0.4 & -0.4 & -0.4\\
  2.05 & 2.05 & 2.05 & 2.05 & 2.05\\
  -2.4 & -2.4 & -2.4 & -2.4 & -2.4\\
  3.75 & 3.75 & 3.75 & 3.75 & 3.75\\
  -1.5 & -1.5 & -1.5 & -1.5 & -1.5\\
\end{pmatrix}
\begin{pmatrix}
  0.069 & 0.258 & 0.351 & 0.646 & 0.972\\
  0.258 & 0.351 & 0.646 & 0.972 & 0.759\\
  0.351 & 0.646 & 0.972 & 0.759 & 0.486\\
  0.646 & 0.972 & 0.759 & 0.486 & 0.295\\
  0.972 & 0.759 & 0.486 & 0.295 & 0.079\\
\end{pmatrix}
\begin{pmatrix}
  -0.383 & 3.86 & -3.277 & 1.717 & -0.214\\
  1.117 & 7.808 & 0.104 & 3.683 & 2.1